# Step, 연산자 오버로드(1)

Variable 인스턴스 a와 b가 있을때, y = a * b 처럼 사용하고 싶다.

이전 Variable 코드를 가져온다.

In [1]:
import numpy as np
import weakref

class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):                            # 입력받는 데이터가 ndarray 구조가 아니면 오류 발생
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data        # 데이터 선언
        self.name = name
        self.grad = None        # 미분값 선언
        self.creator = None     # 이 데이터가 어디출신인지, 어느 공장에서 만들어졌는지 표기
        self.generation = 0     # 세대를 기록하는 변수
    
    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):    # 생성자 = 공장 = 함수
        self.creator = func 
        self.generation = func.generation + 1   # 세대를 기록한다(부모 세대 + 1)

    def cleargrad(self):
        self.grad = None                        # 미분값 초기화

    def backward(self, retain_grad=False):      # 추가되는곳 : 
        if self.grad is None:
            self.grad = np.ones_like(self.data)     # 미분값이 없으면 모두 1로 구성된 행렬

        funcs = []                      # <-- 바뀐부분
        seen_set = set()                # 집합은 중복을 막는다.

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)              # DezZero 함수 리스트를 세대순으로 정렬하는 역할 
                                            # 그결과 funcs.pop()은 자동으로 세대가 가장 큰 DeZero 함수 순으로 꺼낸다. 

        while funcs:
            f = funcs.pop()                         # 함수들을 하나씩 뽑는다.
            gys = [output().grad for output in f.outputs]     # 출력변수인 outputs에 담겨있는 미분값(.grad)들을 리스트에 담는다
            gxs = f.backward(*gys)                          # f의 역전파를 호출한다. *를 붙혀 리스트를 풀면서 넣어준다.(리스트 언팩)
            if not isinstance(gxs, tuple):                  # gxs가 튜플이 아니면 튜플로 변환한다.
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):                # gxs와 f.inputs의 각 원소는 서로 대응 관계
                if x.grad is None:
                    x.grad = gx                             # 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장
                else:
                    x.grad = x.grad + gx    # x.grad가 None이 아니라 기존에 가지고 있는 값이 있다면 가지고 있는 값에 gx를 추가로 더한다.

                if x.creator is not None:
                    add_func(x.creator)      # <-- 바뀐부분, 수정전: funcs.append(x.creator) 출처가 있는 데이터를 add_funcs에 넣는다.
            
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None 

Function 코드도 가져온다.

In [2]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)      # 스칼라이면 array로 바꿔서 리턴
    return x                    # 스칼라 아니면(array이면) 그대로 리턴


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]                       # 가변길이 인수를 다루기위해, 변수를 리스트에 담아 취급
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]       # 가변길이 입력이므로 가변길이 출력을 리스트로 담는다

        self.generation = max([x.generation for x in inputs])   # inputs의 generation중에 가장 큰것

        for output in outputs:
            output.set_creator(self)                     # 각각의 output들이 어디 출신 변수인지 정해짐, 자신이 창조자라고 원산지 표시를 함

        self.inputs = inputs
        #self.outputs = outputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]  # 리스트의 원소가 하나라면 첫번째 원소를 반환한다, 해당 변수를 직접 돌려준다

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


In [3]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

덧셈은 되어있으니 곱셈을 구현해보자

In [4]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1 
        return y 
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data 
        return gy * x1, gy * x0

def mul(x0, x1):
    return Mul()(x0,x1)

In [5]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a,b),c)
y.backward() 

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


하지만 y = add(mul(a,b),c) 방식은 불편하고 원하던게 아니다. 
y = a*b + c 이런식의 표현을 원한다.

이런 연산자를 사용할 수 있도록 Variable을 확장할때, **연산자 오버로드**(operator overload)를 이용한다.

파이썬에서는 __add__ 와 __mul__ 같은 특수 메서드를 정의함으로써 사용자 지정 함수가 호출되도록 한다.

## 20.2 연산자 오버로드 

곱셈 연산자 *를 오버로드  
곱셈의 특수 메서드는 \_\_mul\_\_(self, other)  
\_\_mul\_\_ 메서드를 정의(구현)하면 *연산자를 사용할때 \_\_mul\_\_ 메서드가 호출된다.

~~~python
class Variable:

    # .. 생략 ..

    def __mul__(self, other):
        return mul(self, other)
~~~
위 코드처럼 \__add\__ 와 \__mul\__ 를 추가하면된다.

~~~python 
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
y = a*b
print(y)
~~~



더 간단하게 구현하기 

In [6]:
Variable.__mul__ = mul 
Variable.__add__ = add

In [7]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = a * b + c
y.backward() 

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


# Step21, 연산자 오버로드(2)

지금까지 Variable 인스턴스 a와 b가 있을 때, a*b, a+b를 수행할수 있었다.  
하지만 a * np.array(2.0), 3 + b (ndarray 인스턴스와 함께 사용, 수치 데이터와 함께 사용) 을 할 수 없다.

Variable 인스턴스와 ndarray 인스턴스  
Variable 인스턴스와 int,float 연산

## 21.1 ndarray와 함께 사용하기 

Variable 인스턴스 a  
a * np.array(2.0) --> ndarray 인스턴스를 자동으로 Variable 인스턴스로 변환 : np.array(2.0) ==> Variable(np.array(2.0))

인수로 주어진 객체를 Variable 인스턴스로 변환해주는 함수 

In [8]:
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj 
    return Variable(obj)            # 인자로 받는 객체 인스턴스가 Variable이면 그대로 리턴, 아니면 Variable 씌어서 리턴

In [9]:
class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]           # 추가한 부분
        xs = [x.data for x in inputs]                       # 가변길이 인수를 다루기위해, 변수를 리스트에 담아 취급
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]       # 가변길이 입력이므로 가변길이 출력을 리스트로 담는다

        self.generation = max([x.generation for x in inputs])   # inputs의 generation중에 가장 큰것

        for output in outputs:
            output.set_creator(self)                     # 각각의 output들이 어디 출신 변수인지 정해짐, 자신이 창조자라고 원산지 표시를 함

        self.inputs = inputs
        #self.outputs = outputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]  # 리스트의 원소가 하나라면 첫번째 원소를 반환한다, 해당 변수를 직접 돌려준다

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [10]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0,x1):
    x1 = as_array(x1)
    return Add()(x0,x1)


class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1 
        return y 
    
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data 
        return gy * x1, gy *x0

def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)

~~~python 
class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]           # 추가한 부분
        xs = [x.data for x in inputs]                       # 가변길이 인수를 다루기위해, 변수를 리스트에 담아 취급
~~~

인수 inputs에 담긴 각각의 원소 x를 Variable 인스턴스로 반환  
따라서 ndarray 인스턴스가 주어지면 Variable 인스턴스로 반환  

그러면 이후 처리는 모든 변수가 Variable 인스턴스인 상태로 진행

**NOTE_** 모든 함수(연산)는 Function 클래스를 상속하므로 실제 연산은 Function 클래스의 \__call\__ 메서드에서 이루어진다.  
따라서 \__call\__ 메서드에 가한 수정은 DeZero에서 사용하는 모든 함수에 적용된다.



In [11]:
x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

variable(5.0)


## 21.2 float, int

In [12]:
def add(x0,x1):
    x1 = as_array(x1)
    return Add()(x0,x1)

In [13]:
x = Variable(np.array(2.0))
y = x + 3.0
print(y)

variable(5.0)


### 하지만  
x = Variable(np.array(2.0))  
y = 3.0 + x 인 경우는?  
y = np.array(3.0) + x 인 경우는?

## 21.3 문제점 1: 좌항이 float나 int인 경우

y = 2.0 * x 인 경우 오류 발생과정 

1. 연산자 왼쪽에 있는 2.0의 \__mul\__ 메서드를 호출하려 시도한다. 
2. 하지만 2.0은 float 타입이므로 \__mul\__ 메서드가 구현되어 있지 않다.
3. 다음은 * 연산자 오른쪽에 있는 x의 특수 메서드를 호출하려 시도한다.  
4. x가 오른쪽에 있기 때문에 ( \__mul\__ 대신 )  \__rmul\__ 메서드를 호출하려 시도한다.  
5. 하지만 Variable 인스턴스에는 \__rmul\__ 메서드가 구현되어 있지 않다. 

따라서 \__rmul\__ 메서드를 구현해준다.

In [14]:
Variable.__add__ = add 
Variable.__radd__ = add 
Variable.__mul__ = mul 
Variable.__rmul__ = mul 

In [15]:
x = Variable(np.array(2.0))
y = 1.0 + x 
print(y)

variable(3.0)


In [16]:
x = Variable(np.array(2.0))
y = 3.0 * x + 1.0 
print(y)

variable(7.0)


## 21.4 문제점 2: 좌항이 ndarray 인스턴스인 경우

In [17]:
x = Variable(np.array([1.0]))
y = np.array([2.0]) + x 
print(y)

[variable([3.])]


~~~python
x = Variable(np.array([1.0]))
y = np.array([2.0]) + x 
~~~

위와 같은 경우  
좌항은 ndarray 인스턴스이고  
우항은 Variable 인스턴스  

좌항인 ndarray 인스턴스의 \__add\__ 메서드가 호출된다.  
하지만 우리는 우항인 Variable 인스턴스의 \__radd\__ 메서드가 호출되길 원한다.

**'연산자 우선순위'** 지정 해야한다.
Variable 인스턴스 속성에 \__array_priority\__ 를 추가하고 그 값을 큰 정수로 설정해야한다.

~~~python 
class Variable:
    __array_priority__ = 200
~~~

In [18]:
import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    __array_priority__ = 200

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):                            # 입력받는 데이터가 ndarray 구조가 아니면 오류 발생
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data        # 데이터 선언
        self.name = name
        self.grad = None        # 미분값 선언
        self.creator = None     # 이 데이터가 어디출신인지, 어느 공장에서 만들어졌는지 표기
        self.generation = 0     # 세대를 기록하는 변수
    
    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):    # 생성자 = 공장 = 함수
        self.creator = func 
        self.generation = func.generation + 1   # 세대를 기록한다(부모 세대 + 1)

    def cleargrad(self):
        self.grad = None                        # 미분값 초기화

    def backward(self, retain_grad=False):      # 추가되는곳 : 
        if self.grad is None:
            self.grad = np.ones_like(self.data)     # 미분값이 없으면 모두 1로 구성된 행렬

        funcs = []                      # <-- 바뀐부분
        seen_set = set()                # 집합은 중복을 막는다.

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)              # DezZero 함수 리스트를 세대순으로 정렬하는 역할 
                                            # 그결과 funcs.pop()은 자동으로 세대가 가장 큰 DeZero 함수 순으로 꺼낸다. 

        while funcs:
            f = funcs.pop()                         # 함수들을 하나씩 뽑는다.
            gys = [output().grad for output in f.outputs]     # 출력변수인 outputs에 담겨있는 미분값(.grad)들을 리스트에 담는다
            gxs = f.backward(*gys)                          # f의 역전파를 호출한다. *를 붙혀 리스트를 풀면서 넣어준다.(리스트 언팩)
            if not isinstance(gxs, tuple):                  # gxs가 튜플이 아니면 튜플로 변환한다.
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):                # gxs와 f.inputs의 각 원소는 서로 대응 관계
                if x.grad is None:
                    x.grad = gx                             # 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장
                else:
                    x.grad = x.grad + gx    # x.grad가 None이 아니라 기존에 가지고 있는 값이 있다면 가지고 있는 값에 gx를 추가로 더한다.

                if x.creator is not None:
                    add_func(x.creator)      # <-- 바뀐부분, 수정전: funcs.append(x.creator) 출처가 있는 데이터를 add_funcs에 넣는다.
            
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None 


def as_variable(obj):
    if isinstance(obj, Variable):
        return obj 
    return Variable(obj)            # 인자로 받는 객체 인스턴스가 Variable이면 그대로 리턴, 아니면 Variable 씌어서 리턴


def as_array(x):
    if np.isscalar(x):
        return np.array(x)      # 스칼라이면 array로 바꿔서 리턴
    return x                    # 스칼라 아니면(array이면) 그대로 리턴


class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]           # 추가한 부분
        xs = [x.data for x in inputs]                       # 가변길이 인수를 다루기위해, 변수를 리스트에 담아 취급
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]       # 가변길이 입력이므로 가변길이 출력을 리스트로 담는다

        self.generation = max([x.generation for x in inputs])   # inputs의 generation중에 가장 큰것

        for output in outputs:
            output.set_creator(self)                     # 각각의 output들이 어디 출신 변수인지 정해짐, 자신이 창조자라고 원산지 표시를 함

        self.inputs = inputs
        #self.outputs = outputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]  # 리스트의 원소가 하나라면 첫번째 원소를 반환한다, 해당 변수를 직접 돌려준다

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)


class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)


Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

In [19]:
x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

variable(5.0)


In [20]:
x = Variable(np.array(2.0))
y = np.array(3.0) + x
print(y)

variable(5.0)


In [21]:
y = x + 3.0
print(y)

y = 3.0 * x + 1.0
print(y)

variable(5.0)
variable(7.0)


# Step 22, 연산자 오버로드(3)

뺄셈, 나눗셈, 거듭제곱 연산도 추가해보자.

| 특수 메서드     |   예   | 
| --------------- | ----- |
| __neg__(self) | -self |
| __sub__(self,other) | self - other |
| __rsub__(self,other) | other - self |
| __truediv__(self,other) | self/other |
| __rtruediv__(self,other) | other/self |
| __pow__(self,other) | self**other |

\__neg\__(self) 는 양수 -> 음수, 음수 -> 양수, 부호 연산자, 단항 연산자 

나머지 이항 연산자, 적용 대상이 우항이냐 좌항이냐에 따라 2개의 특수 메서드 중 하나를 선별하여 호출해야한다. 

새로운 연산자를 추가하는 순서 
- 1 Function 클래스를 상속하여 원하는 함수 클래스를 구현한다.(예:Mul 클래스)
- 2 파이썬 함수로 사용할 수 있도록 한다(예:mul 함수)
- 3 Variable 클래스의 연산자를 오버로드한다(예:Variable.\__mul\__ = mul)




## 22.1 음수(부호 변환)

음수의 미분은 y = -x 일떄, $$\frac{\partial y}{\partial x} = -1$$  
따라서 역전파는 상류(출력 쪽)에서 전해지는 미분에 -1을 곱하여 하류로 흘려보내 주면 된다.

In [22]:
from DeZero import Variable
import weakref
import numpy as np
import contextlib

ModuleNotFoundError: No module named 'DeZero'

In [23]:
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj 
    return Variable(obj)            # 인자로 받는 객체 인스턴스가 Variable이면 그대로 리턴, 아니면 Variable 씌어서 리턴


def as_array(x):
    if np.isscalar(x):
        return np.array(x)      # 스칼라이면 array로 바꿔서 리턴
    return x          

class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]           # 추가한 부분
        xs = [x.data for x in inputs]                       # 가변길이 인수를 다루기위해, 변수를 리스트에 담아 취급
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]       # 가변길이 입력이므로 가변길이 출력을 리스트로 담는다

        self.generation = max([x.generation for x in inputs])   # inputs의 generation중에 가장 큰것

        for output in outputs:
            output.set_creator(self)                     # 각각의 output들이 어디 출신 변수인지 정해짐, 자신이 창조자라고 원산지 표시를 함

        self.inputs = inputs
        #self.outputs = outputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]  # 리스트의 원소가 하나라면 첫번째 원소를 반환한다, 해당 변수를 직접 돌려준다

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [24]:
class Neg(Function):
    def forward(self, x):
        return -x 
    
    def backward(self, gy):
        return -gy 

def neg(x):
    return Neg()(x)

Variable.__neg__ = neg

In [25]:
x = Variable(np.array(2.0))
y = -x 
print(y)

variable(-2.0)


## 22.2 뺄셈 

뺄셈의 미분은 y = x0 - x1 일떄, $$\frac{\partial y}{\partial x0} = 1, \frac{\partial y}{\partial x1} = -1$$  
따라서 역전파는 상류(출력 쪽)에서 전해지는 미분값에 1을 곱한값이 x0의 미분 결과  
-1을 곱한값이 x1의 미분 결과

x0,x1이 Variable 인스턴스  
y = x0 - x1 계산 수행 가능  
y = 2.0 - x1 같은 코드는 X   
x의 \__rsub\__ 메서드가 호출  

\__rsub\__(self,other) : 우항인 x가 인수 self에 전달된다.

**CAUTION_** 덧셈과 곱셈은 좌항과 우항의 순서를 바꿔도 상관없다  
하지만 뺄셈에서는 좌우를 구별해야 한다(x0 - x1 과 x1 - x0은 다르다)  
따라서 우항을 대상으로 했을때 적용할 함수인 rsub(x0, x1)을 별도로 준비

In [26]:
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1 
        return y 
    
    def backward(self, gy):
        return gy, -gy

def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

def rsub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x1, x0)

Variable.__sub__ = sub
Variable.__rsub__ = rsub

In [27]:
x = Variable(np.array(2.0))
y1 = 2.0 - x 
y2 = x - 1.0 
print(y1)
print(y2)

variable(0.0)
variable(1.0)


## 22.3 나눗셈 

나눗셈의 미분은 y = x0/x1 일때, $$\frac{\partial y}{\partial x0} = 1/x1, \frac{\partial y}{\partial x1} = -x0/(x1)^2$$  


In [28]:
class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1 
        return y 
    
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data 
        gx0 = gy / x1 
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1 
    
def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0,x1)

def rdiv(x0, x1):
    x1 = as_array(x1)
    return Div()(x1,x0)

Variable.__truediv__ = div 
Variable.__rtruediv__ = rdiv

## 22.4 거듭제곱

거듭제곱은 $$y = x^c$$ 로 표현  
거듭제곱의 미분 : $$\frac{\partial y}{\partial x} = cx^{c-1}$$  
따라서 역전파는 상류(출력 쪽)에서 전해지는 미분값에 1을 곱한값이 x0의 미분 결과  
-1을 곱한값이 x1의 미분 결과

In [29]:
class Pow(Function):
    def __init__(self, c):
        self.c = c 
    
    def forward(self, x):
        y = x ** self.c 
        return y 
    
    def backward(self, gy):
        x = self.inputs[0].data 
        c = self.c 
        gx = c * x ** (c-1) * gy 
        return gx 

def pow(x,c):
    return Pow(c)(x)

Variable.__pow__ = pow

In [30]:
x = Variable(np.array(2.0))
y = x ** 3
print(y)

variable(8.0)


a // b, a % b, a += 1, a += -2